In [29]:
# import packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, RidgeCV, PoissonRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [6]:
# read data
bikes = pd.read_csv('data/finaldata.csv')

In [7]:
bikes.head()

,date,numbikes,log_numbikes,year,month,week,day,dayofweek,dayofyear,sin_doy,cos_doy,sin_dow,cos_dow,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,holiday,awnd,prcp,snow,snwd,tmax,tmin,awnd_log,prcp_log,snow_log,snwd_log
0,2011-01-01,959,6.865891,2011,1,52,1,5,1,0.017213,0.999852,-0.974928,-0.222521,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.2,0.3,0.0,0.0,14.4,1.1,1.435085,0.262364,0.0,0.0
1,2011-01-02,781,6.660575,2011,1,52,2,6,2,0.034422,0.999407,-0.781831,0.623490,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.9,1.5,0.0,0.0,15.0,1.1,1.774952,0.916291,0.0,0.0
2,2011-01-03,1301,7.170888,2011,1,1,3,0,3,0.051620,0.998667,0.000000,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.6,0.0,0.0,0.0,5.0,-2.2,1.722767,0.000000,0.0,0.0
3,2011-01-04,1536,7.336937,2011,1,1,4,1,4,0.068802,0.997630,0.781831,0.623490,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.9,0.0,0.0,0.0,6.1,-3.9,1.360977,0.000000,0.0,0.0
4,2011-01-05,1571,7.359468,2011,1,1,5,2,5,0.085965,0.996298,0.974928,-0.222521,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.8,0.0,0.0,0.0,6.1,-0.6,1.568616,0.000000,0.0,0.0


In [34]:
# define list of features for regression analysis
calendar_features = [f'dow_{i+1}' for i in range(0,6)] + \
                    [f'month_{i+1}' for i in range(1,12)] + \
                    ['sin_doy', 'cos_doy', 'sin_dow', 'cos_dow', 'holiday', 'year']
weather_features = ['awnd_log', 'prcp_log', 'snow_log',	'snwd_log', 'tmax', 'tmin']

In [81]:
def reg_predict(
    df,
    features,
    train_year_end = 2018, # training year from 2011 up until year specified here
    test_year = 2019, # test year must be after train_year_end, must be between 2011 and 2019
    model = 'linear', # choice of 'linear', 'polynomial', 'ridge', 'poisson'
    scaling = True, # default scaling (standardization) of x variables
    logbikes = False, # default y variable is not log transformed
    degree = 2, # quadratic polynomial is default for polynomial and ridge regression
    alphas = [0.01,0.1,1,10,100] # regularization grid for ridge regression
):
    # define train/test years
    df_train = df[df['year'] <= train_year_end]
    df_test = df[df['year'] == test_year]

    # define X, y for train/test sample
    X_train = df_train[features]
    X_test = df_test[features]
    y_train = df_train['numbikes']
    y_test = df_test['numbikes']

    # scaling standardization for features
    if scaling:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    # log choice for number of bikes
    if logbikes:
        y_train = np.log(y_train)
        y_test = np.log(y_test)
    
    # choice for regression models
    if model == 'linear':
        regr = LinearRegression()
    elif model == 'polynomial':
        regr = make_pipeline(
            PolynomialFeatures(degree=degree),
            LinearRegression()
            )
    elif model == 'ridge':
        regr = make_pipeline(
            PolynomialFeatures(degree=degree),
            RidgeCV(alphas=alphas, cv=TimeSeriesSplit())
            )
    elif model == 'poisson':
        regr = PoissonRegressor()
    else:
        assert False, f'Unknown model {model}'
    
    # fitting and prediction
    regr.fit(X_train, y_train)
    y_pred_train = regr.predict(X_train)
    y_pred = regr.predict(X_test)
    
    # print r2_score and mape for train/test of model
    print('----------------------')
    print(f'{model} regression')
    print('----------------------')
    print(f'Train MAPE: {mean_absolute_percentage_error(y_train, y_pred_train).round(3)}')
    print(f'Test MAPE: {mean_absolute_percentage_error(y_test, y_pred).round(3)}')
    print('----------------------')

In [75]:
# baseline model 
reg_predict(df=bikes, features=calendar_features)

----------------------
linear regression
----------------------
Train MAPE: 0.476
Test MAPE: 0.51
----------------------


In [76]:
# use logbikes as y variable on baseline model
reg_predict(df=bikes, features=calendar_features, logbikes=True)

----------------------
linear regression
----------------------
Train MAPE: 0.034
Test MAPE: 0.043
----------------------


In [77]:
# use logbikes and polynomial regression degree 2
reg_predict(df=bikes, features=calendar_features, logbikes=True, model='polynomial')

----------------------
polynomial regression
----------------------
Train MAPE: 0.027
Test MAPE: 0.029
----------------------


In [82]:
# use logbikes and ridge regression
reg_predict(df=bikes, features=calendar_features, logbikes=True, model='ridge')

----------------------
ridge regression
----------------------
Train MAPE: 0.027
Test MAPE: 0.029
----------------------


In [79]:
# use logbikes and poisson regression
reg_predict(df=bikes, features=calendar_features, logbikes=True, model='poisson')

----------------------
poisson regression
----------------------
Train MAPE: 0.035
Test MAPE: 0.039
----------------------


In [80]:
# Include weather and calendar features
features = [*calendar_features, *weather_features]

In [83]:
# baseline model 
reg_predict(df=bikes, features=features)

----------------------
linear regression
----------------------
Train MAPE: 0.267
Test MAPE: 0.436
----------------------


In [84]:
reg_predict(df=bikes, features=features, logbikes=True)

----------------------
linear regression
----------------------
Train MAPE: 0.027
Test MAPE: 0.043
----------------------
